In [1]:
import os
import numpy as np
from EnsemblePursuit.EnsemblePursuit import EnsemblePursuit
import matplotlib.pyplot as plt

from keras.applications.vgg16 import VGG16
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(300, 300, 3))

Using TensorFlow backend.


58892288/58889256 [==============================] - 10s 0us/step


In [21]:
# example of using the vgg16 model as a feature extraction model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
from pickle import dump
from tensorflow.keras import Input

fname='/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/kay_images.npz'
with np.load(fname) as dobj:
    dat = dict(**dobj)
# load an image from file
#image = load_img('dog.jpg', target_size=(224, 224))
import cv2

image=dat['stimuli'][0,:,:]
image=cv2.resize(image,(224,224))
print(image.shape)
# convert the image pixels to a numpy array
image = img_to_array(image)
#img_input = Input(shape=(img_size_target,img_size_target,1))
image=np.repeat(image,3,axis=2)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1],image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# load model
model = VGG16()
# remove the output layer
model.layers.pop()
input_tensor=Input(shape=(128, 128, 3))
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
# get extracted features
features = model.predict(image)
print(features.shape)
# save to file
dump(features, open('dog.pkl', 'wb'))

(224, 224)
(1, 4096)
